In [1]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer
import gc
import shutil

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-22 23:04:20.007 | INFO     | train:<module>:5 - Loading dependencies - Torch...
2024-11-22 23:04:20.032 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-22 23:04:20.033 | INFO     | train:<module>:25 - Loading dependencies - Project...


In [2]:
params = LMHyperParams(
    base_model_checkpoint="Qwen/Qwen2.5-0.5B",
    warmup_steps_count=10,
    tuning_type="dpo",
    train_batch_size=1,
    gradient_accumulation_steps=1,
    optimizer="AdamW",
)

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [3]:
model = AutoLMFineTuner(params)
model.to('cuda')

2024-11-22 23:04:25.223 | INFO     | model.causal_lm:__init__:69 - Using DPO with LoraConfig: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=256, target_modules='all-linear', lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))


AutoLMFineTuner(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): Qwen2ForCausalLM(
        (model): Qwen2Model(
          (embed_tokens): Embedding(151936, 896)
          (layers): ModuleList(
            (0-23): 24 x Qwen2DecoderLayer(
              (self_attn): Qwen2SdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=896, out_features=896, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=896, out_features=256, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=256, out_features=896, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_vec

In [4]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

4
Fri Nov 22 23:03:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:53:00.0 Off |                    0 |
| N/A   25C    P0             110W / 700W |  48869MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+

In [4]:
data_module = UltraFeedbackDataModule(2, model.tokenizer, 1024, 10)
shutil.rmtree(data_module.cache_dir)
data_module.cpu_count = 12
data_module.setup("fit")

2024-11-22 23:04:39.767 | INFO     | dataset.squad:setup:219 - Loading dataset for stage fit
2024-11-22 23:04:41.369 | INFO     | dataset.squad:setup:230 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback
num_proc must be <= 9. Reducing num_proc to 9 for dataset of size 9.
Parameter 'function'=<bound method UltraFeedbackDataModule.process_samples_batch of <dataset.squad.UltraFeedbackDataModule object at 0x155107dbfa50>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map (num_proc=9): 100%|██████████| 9/9 [00:01<00:00,  7.91 examples/s]
num_

In [17]:
data_module.precompute_reference_logprobs(model)

Precomputing reference logprobs: 100%|██████████| 5/5 [00:17<00:00,  3.45s/it]


[{'chosen_ref_log_probs': tensor([[[ 9.7230,  9.1477, 14.8778,  ..., -2.8170, -2.8184, -2.8170],
         [10.6167,  9.2509, 16.3828,  ..., -2.6728, -2.6745, -2.6729],
         [10.8379,  9.5980, 15.6573,  ..., -2.6257, -2.6273, -2.6258],
         ...,
         [ 5.8885,  0.5392,  4.4016,  ..., -4.1522, -4.1528, -4.1523],
         [ 9.5476,  7.4200,  9.7727,  ..., -2.6856, -2.6859, -2.6855],
         [-0.4611,  2.9680,  0.1975,  ...,  4.5102,  4.5104,  4.5102]],

        [[13.1853, 12.3194, 16.9904,  ..., -2.6882, -2.6901, -2.6882],
         [13.1143, 12.2432, 17.3645,  ..., -2.6545, -2.6562, -2.6546],
         [12.1844, 12.3025, 16.6598,  ..., -2.6709, -2.6724, -2.6709],
         ...,
         [ 5.7268, -1.3190,  6.1402,  ..., -3.7019, -3.7019, -3.7020],
         [ 8.8550,  6.0457, 10.3195,  ..., -2.2063, -2.2060, -2.2063],
         [-3.4919, -4.1036, -1.2868,  ...,  6.0157,  6.0164,  6.0156]]]), 'rejected_ref_log_probs': tensor([[[ 9.7230,  9.1477, 14.8778,  ..., -2.8170, -2.8184, -2

Saving the dataset (5/23 shards): 100%|██████████| 5/5 [00:09<00:00,  1.99s/ examples]


IndexError: Index 5 out of range for dataset of size 5.

In [ ]:
for batch in data_module.train_dataloader():
    for k, v in batch.items():
        batch[k] = v.to('cuda')
    loss = model._step(batch)
    print(loss)
    break

In [ ]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)
